Imports

In [1]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import helpers.evaluate
from helpers.model_list import model_map

Load in Data

In [2]:
df = pd.read_csv("diabetes_binary_5050split_health_indicators_BRFSS2015.csv")
X = df.drop(columns=["Diabetes_binary"])
y = df["Diabetes_binary"]
df.head()

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,3.0,5.0,30.0,0.0,1.0,4.0,6.0,8.0
1,0.0,1.0,1.0,1.0,26.0,1.0,1.0,0.0,0.0,1.0,...,1.0,0.0,3.0,0.0,0.0,0.0,1.0,12.0,6.0,8.0
2,0.0,0.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,10.0,0.0,1.0,13.0,6.0,8.0
3,0.0,1.0,1.0,1.0,28.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,3.0,0.0,3.0,0.0,1.0,11.0,6.0,8.0
4,0.0,0.0,0.0,1.0,29.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,8.0,5.0,8.0


Explore Data

In [ ]:
#TODO

Clean Data

In [ ]:
#TODO

Split Data

In [3]:
#Transform target
le = LabelEncoder()
y = le.fit_transform(y)

#Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8)

#Scale Data
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#Reassign col names
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)


Train Models

In [7]:
#Import models
models = {name: model_class() for name, model_class in model_map.items()}

total_metrics = {}

#train each model
for name, model in models.items():
    model.train(X_train_scaled, y_train)
    metrics = model.predict(X_test_scaled, y_test)
    total_metrics[name] = metrics
    helpers.evaluate.print_stats(metrics, name)


Decision Tree  Accuracy Score:  0.6534816281783782
Decision Tree  Precision Score:  0.6520266694888348
Decision Tree  Recalle Score:  0.6550538701445988
Decision Tree  F1 Score:  0.6535367643158956
Logistic Regression  Accuracy Score:  0.7468614067970435
Logistic Regression  Precision Score:  0.7345595680053999
Logistic Regression  Recalle Score:  0.7713708534165012
Logistic Regression  F1 Score:  0.7525152992428171
kNearestNeighbors  Accuracy Score:  0.6989249213141422
kNearestNeighbors  Precision Score:  0.6883606612571967
kNearestNeighbors  Recalle Score:  0.724588885738588
kNearestNeighbors  F1 Score:  0.7060103251204309


Report Stats

In [10]:
#Get best metrics
bestModel = ""
bestPerformance = 0
for name, metrics in total_metrics.items():
    if(metrics[0] > bestPerformance):
        bestModel = name
        bestPerformance = metrics[0]

print("Best Model = ", bestModel)
print("Model Accuracy = ", bestPerformance)

Best Model =  Logistic Regression
Model Accuracy =  0.7468614067970435
